In the previous tutorial we created a work queue. The assumption behind a work queue is that each task is delivered to exactly one worker. In this part we'll do something completely different -- we'll deliver a message to multiple consumers.  

To illustrate the pattern, we're going to build a simple logging system. It will consist of two programs -- the first will emit log messages and the second will receive and print them.

Essentially, published log messages are going to be broadcast to all the receivers.

### Exchanges
In previous parts of the tutorial we sent and received messages to and from a queue. Now it's time to introduce the full messaging model in Rabbit.

Let's quickly go over what we covered in the previous tutorials:

A producer is a user application that sends messages.
A queue is a buffer that stores messages.
A consumer is a user application that receives messages.
The core idea in the messaging model in RabbitMQ is that the producer never sends any messages directly to a queue. Actually, quite often the producer doesn't even know if a message will be delivered to any queue at all.

Instead, the producer can only send messages to an exchange. An exchange is a very simple thing. On one side it receives messages from producers and the other side it pushes them to queues. The exchange must know exactly what to do with a message it receives. Should it be appended to a particular queue? Should it be appended to many queues? Or should it get discarded. The rules for that are defined by the exchange type.

There are a few exchange types available: direct, topic, headers and fanout. We'll focus on the last one -- the fanout. Let's create an exchange of that type, and call it logs:

channel.exchange_declare(exchange='logs',exchange_type='fanout')

The fanout exchange is very simple. As you can probably guess from the name, it just broadcasts all the messages it receives to all the queues it knows. And that's exactly what we need for our logger.

### Listing exchanges
To list the exchanges on the server you can run the ever useful rabbitmqctl:

sudo rabbitmqctl list_exchanges

In this list there will be some amq.* exchanges and the default (unnamed) exchange. These are created by default, but it is unlikely you'll need to use them at the moment.

#### The default exchange
In previous parts of the tutorial we knew nothing about exchanges, but still were able to send messages to queues. That was possible because we were using a default exchange, which we identify by the empty string ("").

Recall how we published a message before:

channel.basic_publish(exchange='',
                      routing_key='hello',
                      body=message)
                      
The exchange parameter is the name of the exchange. The empty string denotes the default or nameless exchange: messages are routed to the queue with the name specified by routing_key, if it exists.

### Temporary queues

As you may remember previously we were using queues that had specific names (remember hello and task_queue?). Being able to name a queue was crucial for us -- we needed to point the workers to the same queue. Giving a queue a name is important when you want to share the queue between producers and consumers.

But that's not the case for our logger. We want to hear about all log messages, not just a subset of them. We're also interested only in currently flowing messages not in the old ones. To solve that we need two things.

Firstly, whenever we connect to Rabbit we need a fresh, empty queue. To do it we could create a queue with a random name, or, even better - let the server choose a random queue name for us. We can do this by not supplying the queue parameter to queue_declare:

result = channel.queue_declare()
At this point result.method.queue contains a random queue name. For example it may look like amq.gen-JzTY20BRgKO-HjmUJj0wLg.

Secondly, once the consumer connection is closed, the queue should be deleted. There's an exclusive flag for that:

result = channel.queue_declare(exclusive=True)

### Bindings

We've already created a fanout exchange and a queue. Now we need to tell the exchange to send messages to our queue. That relationship between exchange and a queue is called a binding.

channel.queue_bind(exchange='logs',
                   queue=result.method.queue)
From now on the logs exchange will append messages to our queue.

#### Listing bindings
You can list existing bindings using, you guessed it,

sudo rabbitmqctl list_bindings

**Before running the next cell, run the to receivers first: receive_logs_1, receive_logs_2**

In [4]:
import pika
import sys
pika.__version__

'1.1.0'

In [5]:
#!sudo service rabbitmq-server restart
#!sudo rabbitmqctl status

In [6]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [7]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-oueTZmRTr3lFeNnR4dEr3A	queue	amq.gen-oueTZmRTr3lFeNnR4dEr3A	[]
	exchange	amq.gen-somBXTFBWwemrcgsEjFLdQ	queue	amq.gen-somBXTFBWwemrcgsEjFLdQ	[]
	exchange	hello2	queue	hello2	[]
logs	exchange	amq.gen-oueTZmRTr3lFeNnR4dEr3A	queue	amq.gen-oueTZmRTr3lFeNnR4dEr3A	[]
logs	exchange	amq.gen-somBXTFBWwemrcgsEjFLdQ	queue	amq.gen-somBXTFBWwemrcgsEjFLdQ	[]


In [8]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
    
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)

iter=0
exchangeName = 'logs'

In [9]:
iter=iter+1
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange=exchangeName, exchange_type='fanout')
#channel.queue_declare(queue='task_queue', durable=True)

#send message #for fanout type exchange, routing key/queue is useless
properties = pika.spec.BasicProperties(content_type = "text/plain", delivery_mode = 1)
message = "MessageToEveryOne: "+str(iter)

channel.basic_publish(exchange = exchangeName, routing_key = "", body = message, properties = properties)

#disconnect
connection.close()

**After this, we should see that all workers received the same message**

In [10]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-oueTZmRTr3lFeNnR4dEr3A	queue	amq.gen-oueTZmRTr3lFeNnR4dEr3A	[]
	exchange	amq.gen-somBXTFBWwemrcgsEjFLdQ	queue	amq.gen-somBXTFBWwemrcgsEjFLdQ	[]
	exchange	hello2	queue	hello2	[]
logs	exchange	amq.gen-oueTZmRTr3lFeNnR4dEr3A	queue	amq.gen-oueTZmRTr3lFeNnR4dEr3A	[]
logs	exchange	amq.gen-somBXTFBWwemrcgsEjFLdQ	queue	amq.gen-somBXTFBWwemrcgsEjFLdQ	[]


In [11]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
logs	fanout


**Let us try once more**

In [12]:
iter=iter+1
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange=exchangeName, exchange_type='fanout')
#channel.queue_declare(queue='task_queue', durable=True)

#send message #for fanout type exchange, routing key/queue is useless
properties = pika.spec.BasicProperties(content_type = "text/plain", delivery_mode = 1)
message = "MessageToEveryOne: "+str(iter)

channel.basic_publish(exchange = exchangeName, routing_key = "", body = message, properties = properties)

#disconnect
connection.close()

**After this cell, all workers should have received the second message too**